# SingleLine Playground

This notebook explores some ideas in one-lining Python code while limiting function stack usage.

Warning: lots of hacks ahead!

__Disclaimer__: I realized that this file is a lot less necessary than I initially thought... The target code turned out to be a lot more about using the walrus operator than encoding various stuff with lambda.

## Basic Recursions

Basic recursions in lambda calculus is trivial: you just need a self-reference, usually done with a Y-combinator. However, its counterpart (Z-combinator) in a non-lazy language induces way too many layers of lambda application, which severely reduces the allowed depth of recursion. This project just stupidly uses the walrus operator `:=` to alter the namespace instead of using lambda bindings. This preserves the allowed recursion depth completely.

In [125]:
# source function

def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)


# transformed code

_ = (gcd := (lambda a, b: a if b == 0 else gcd(b, a % b)))

## Mutual Recursions

Just like the case for basic recursions, the typical lambda encoding of `letrec ... in ...` introduces too many function calls. Instead the walrus operator is abused again...

Note that if a function returns `None` in the source code, it will return `None` after transformation even if its return value is never used according to a data-flow analysis. This is done in case another file imports a transformed function and expect it to return `None`. While this isn't true for functions defined inside another function (thus leaving room for optimization by omitting `return None`), this is not currently implemented.

In [126]:
# source functions

def even(a):
    if a > 0:
        print(a)
        odd(a - 1)
        
def odd(a):
    print(a)
    even(a - 1)
    

# transformed code

_ = (
    even := lambda a: ((print(a), odd(a - 1)) if a > 0 else (), None)[-1], # even
    odd := lambda a: (print(a), even(a - 1), None)[-1] # odd
)

## If Statements

`if` statements are 

## For Loops

For loops introduces complication due to the possibility of an early termination or interruption with `break`, `return` or `continue`. "Pure" for loops, i.e. without any interruption, can be written as a list comprehension:

In [127]:
# source code

acc = 0
for i in range(5):
    acc += i
    

# transformed code

_ = (acc := 0, [acc := acc + i for i in range(5)])

## Loops with Interruption

For `for` loops with interruption and `while` loops, a right fold is used to handle early termination and `continue`. This is achieved with `filter` on an infinite generator, paired with a preliminary variable to store the accumulated state. Note that since assignment of a variable outside of the scope of a lambda is not possible, the accumulated state is a bundle of all the variables altered in the `while` loop, along with a flag indicating whether the loop should terminate: `[x1, x2, ..., xn, flag]`.

After the loop ends, all variables in the accumulated variable are unpacked into their original names (only for variables that are used afterwards according to the data flow analysis).

### Accumulator Encoding

As mentioned above, the accumulator for a loop should capture all altered variables in the loop, along with some flags that describes the execution state of the loop. The specific format of an accumulator is defined as:
`[x1, ..., xn, ret_val, has_ret, break_term, while_term]`

- `ret_val` (object) and `has_ret` (boolean) only exists when the loop is in a function and has the possibility of returning a value. `has_ret` describes whether the loop triggered a return statement, and `ret_val` contains the return value.
- `break_term` (boolean) is a termination flag that only exists when the loop has the possibility of encountering a `break` statement.
- `while_term` (boolean) is a termination flag that only exists when the loop is a while loop.

If any one of `has_ret`, `break_term` or `while_term` is `True`, the loop will be terminated. In the AST transformer, the existence of these flags are decided automatically based on an analysis pass on the source code of the loop (e.g., if a `while` loop does not have `break` or `return` in it then those flags will not be generated).

While loop:

In [128]:
# source code

x = 0
acc = 0
while x // 500 < 100:
    acc += x
    x += 1

print(acc)


# transformed code

# import can be inlined, but now we're just demonstrating
from itertools import count

_ = (
    # intermediate state (accumulator)
    # note that only the `while_term` exist
    a := [x := 0, acc := 0, not (x // 500 < 100)],
    
    next(filter(
        lambda _: True if any(a[-1 :]) else
        (
            a.__setitem__(1, a[1] + a[0]), # acc += 1
            a.__setitem__(0, a[0] + 1), # x += 1
            a.__setitem__(-1, not (a[0] // 500 < 100)), # updates the while loop termination flag
            False # does not stop the `filter`
        )[-1],
        count() # `while` maps over an infinite generator
    )),
    
    acc := a[1], # unpacking the accumulator
    print(acc)
)

1249975000
1249975000


`break` and `continue` are implemented by exploiting the short-wiring of the `... if ... else ...` expression.

Note that `break` and `continue` generates the same code, except `break` sets the `break_term` flag.

A control flow graph is created for this purpose during the compilation process.

In [129]:
# source code

odds = []
for i in range(100):
    if i & 1 != 0: continue
    if i > 50: break
        
    odds.append(i)
    
print(odds)


# transformed code

from itertools import count

_ = (
    odds := [], # `odds` is never reassigned a value in the `for`, thus not packed in accumulator state
    
    # intermediate state, note that the only flag is `break_term`
    a := [False],
    
    next(filter(
        
        # `break_term` interruption
        lambda i: True if any(a[-1 :]) else
        (
            # `continue_flag` of a branching statement
            c_flag := False,
            
            # `if i & 1 != 0: continue`
            (c_flag := True) if i & 1 != 0 else (),
            
            # rest of program after `if i & 1 != 0: continue`, else returns `False` to continue
            False if c_flag else
            (
                # `continue_flag` of the `if i > 50` statement
                c_flag := False,
                
                # `if i > 50: break`, note that `break_term` is set
                (c_flag := (a.__setitem__(-1, True), True)[-1]) if i > 50 else (),
                
                False if c_flag else odds.append(i),
                
                # next iteration
                False
            )[-1]
        )[-1],
        range(100)
    )),
    print(odds)
)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
